In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['figure.autolayout'] = False

In [2]:
# Turn off/on the presentation toolbar at the bottom of the notebook.

In [3]:
%%javascript
$('.nbp-app-bar').toggle()

<IPython.core.display.Javascript object>

## Read in the Data

In [4]:
path = '../kaggle_data/input/'
raw = pd.read_csv(path+'weather.csv')

In [5]:
raw.head(6)

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9
5,2,2007-05-03,67,48,58,M,40,50,7,0,...,HZ,M,M,M,0.00,29.46,30.12,12.9,6,13.2


In [6]:
print(raw.shape)
print(raw.columns)

(2944, 22)
Index(['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'Depth',
       'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel',
       'ResultSpeed', 'ResultDir', 'AvgSpeed'],
      dtype='object')


In [7]:
# What do rows mean? There are two rows per day.
print(len(raw))
print(1472*2)
raw['Date'].nunique()

2944
2944


1472

In [8]:
# What is the second, empty row? It's station #2.
raw[['Station', 'Date', 'Sunrise', 'Sunset']].head(6)

,Station,Date,Sunrise,Sunset
0,1,2007-05-01,0448,1849
1,2,2007-05-01,-,-
2,1,2007-05-02,0447,1850
3,2,2007-05-02,-,-
4,1,2007-05-03,0446,1851
5,2,2007-05-03,-,-


In [9]:
raw.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [10]:
# Looks like two readings were taken every day, but didn't include sunrise and sunset. All other readings are there.
raw[['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'CodeSum', 'Depth',
       'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel']].head(4)

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel
0,1,2007-05-01,83,50,67,14,51,56,0,2,,0,M,0.0,0.00,29.10,29.82
1,2,2007-05-01,84,52,68,M,51,57,0,3,,M,M,M,0.00,29.18,29.82
2,1,2007-05-02,59,42,51,-3,42,47,14,0,BR,0,M,0.0,0.00,29.38,30.09
3,2,2007-05-02,60,43,52,M,42,47,13,0,BR HZ,M,M,M,0.00,29.44,30.08


In [11]:
# I don't plan to use sunrise/sunset (too correlated with time of year), so we can just groupby Date.
# I'll take the maximum value from each of the two readings.
weather=raw.groupby('Date').max().reset_index()
weather.shape

(1472, 22)

## Creating Binary Variables

In [12]:
# Make a binary variable to track rainy days:
weather['rainy']=weather['CodeSum'].str.contains('RA').astype(int)
weather[['rainy', 'CodeSum']].tail()

,rainy,CodeSum
1467,1,RA
1468,1,RA
1469,0,
1470,1,RA
1471,1,RA SN BR


In [13]:
weather['DewPoint'].describe()

count    1472.000000
mean       54.015625
std        10.677290
min        23.000000
25%        47.000000
50%        55.000000
75%        62.000000
max        75.000000
Name: DewPoint, dtype: float64

In [14]:
# A higher dew point means there will be more moisture in the air
print(weather[weather['DewPoint']>53]['DewPoint'].value_counts().sort_values(ascending=False).head())
weather['dry']=1
weather.loc[weather['DewPoint']>=53, 'dry']=0
print(weather['dry'].value_counts())
weather[['dry', 'DewPoint']].head()

54    63
60    59
56    58
59    57
61    55
Name: DewPoint, dtype: int64
0    876
1    596
Name: dry, dtype: int64


,dry,DewPoint
0,1,51
1,1,42
2,1,40
3,1,42
4,1,39


In [15]:
# We don't need to change the temperature variable:
weather['Tmax'].describe()

count    1472.000000
mean       76.740489
std        11.433218
min        42.000000
25%        69.000000
50%        79.000000
75%        85.000000
max       104.000000
Name: Tmax, dtype: float64

In [16]:
# WetBulb has 4 missing values. 
print(weather[weather['WetBulb']=='M']['WetBulb'].count()) # How many M's are there?
print(weather.loc[weather['WetBulb']!='M']['WetBulb'].mode()) # What's the mode when it's not M?
weather.loc[weather['WetBulb']=='M', 'WetBulb']='65' # Replace M with the mode.
weather['WetBulb']=weather['WetBulb'].astype(int) # Now convert to an integer

4
0    65
dtype: object


In [17]:
# If a Dry-Wet difference is greater than “5” than that day is marked as “1” 
weather['diff']=weather['WetBulb'].astype(int) - weather['DewPoint'].astype(int)
print(len(weather.loc[weather['diff']>5]))
weather['Dry_Wet_Diff']=0
weather.loc[weather['diff']>5, 'Dry_Wet_Diff']=1
weather['Dry_Wet_Diff'].value_counts()

719


0    753
1    719
Name: Dry_Wet_Diff, dtype: int64

In [18]:
# Slowdown: All the days where “ResultSpeed” value is less than “11” 
weather['Slowdown']=0
weather.loc[weather['ResultSpeed']<11, 'Slowdown']=1
weather['Slowdown'].value_counts()

1    1236
0     236
Name: Slowdown, dtype: int64

In [19]:
# There are three rows where AvgSpeed is M.
print(weather[weather['AvgSpeed']=='M']['AvgSpeed'].count())
print(weather.loc[weather['AvgSpeed']!='M']['AvgSpeed'].mode()) # What's the mode when it's not M?
weather.loc[weather['AvgSpeed']=='M', 'AvgSpeed']='9.6' # Replace M with the mode.
weather['AvgSpeed']=weather['AvgSpeed'].astype(float) # Now convert to an integer

3
0    9.6
dtype: object


In [20]:
# Speedy: All the days where  “AvgSpeed” value is greater than “10”
weather['Speedy']=0
weather.loc[weather['AvgSpeed']>10, 'Speedy']=1
weather['Speedy'].value_counts()

0    1099
1     373
Name: Speedy, dtype: int64

In [21]:
weather.columns

Index(['Date', 'Station', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'Depth',
       'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel',
       'ResultSpeed', 'ResultDir', 'AvgSpeed', 'rainy', 'dry', 'diff',
       'Dry_Wet_Diff', 'Slowdown', 'Speedy'],
      dtype='object')

## Did it rain last week?

In [22]:
rlist=[]
for x in range(1,8):
    digit=str(x)
    weather['rainy-'+digit] = weather['rainy'].shift(x)
    rlist.append('rainy-'+ digit)
weather['avg_rain1'] = weather[rlist].mean(axis=1)
weather['avg_rain1'].describe()

count    1471.000000
mean        0.402672
std         0.211470
min         0.000000
25%         0.285714
50%         0.428571
75%         0.571429
max         1.000000
Name: avg_rain1, dtype: float64

## Did it rain two weeks ago?

In [23]:
rlist=[]
for x in range(8,15):
    digit=str(x)
    weather['rainy-'+digit] = weather['rainy'].shift(x)
    rlist.append('rainy-'+ digit)
weather['avg_rain2'] = weather[rlist].mean(axis=1)
weather['avg_rain2'].describe()

count    1464.000000
mean        0.403719
std         0.211185
min         0.000000
25%         0.285714
50%         0.428571
75%         0.571429
max         1.000000
Name: avg_rain2, dtype: float64

## Last week's dryness

In [33]:
# This variable is a 0/1 indicator showing whether the difference between wetbulb and dewpoint.
# If a difference is greater than “5” than that day is marked as “1” 
# SOURCE: https://www.kaggle.com/anjanaagrawal/data-exploration-worked-for-me/comments
weather['Dry_Wet_Diff'].describe()

count    1472.000000
mean        0.488451
std         0.500036
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: Dry_Wet_Diff, dtype: float64

In [24]:
dlist=[]
for x in range(1,8):
    digit=str(x)
    weather['Dry_Wet_Diff-'+digit] = weather['Dry_Wet_Diff'].shift(x)
    dlist.append('Dry_Wet_Diff-'+ digit)
weather['avg_dry1'] = weather[dlist].mean(axis=1)

## Two week's dryness

In [25]:
dlist=[]
for x in range(8,15):
    digit=str(x)
    weather['Dry_Wet_Diff-'+digit] = weather['Dry_Wet_Diff'].shift(x)
    dlist.append('Dry_Wet_Diff-'+ digit)
weather['avg_dry2'] = weather[dlist].mean(axis=1)

## Export to csv

In [26]:
weather.columns

Index(['Date', 'Station', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'Depth',
       'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel',
       'ResultSpeed', 'ResultDir', 'AvgSpeed', 'rainy', 'dry', 'diff',
       'Dry_Wet_Diff', 'Slowdown', 'Speedy', 'rainy-1', 'rainy-2', 'rainy-3',
       'rainy-4', 'rainy-5', 'rainy-6', 'rainy-7', 'avg_rain1', 'rainy-8',
       'rainy-9', 'rainy-10', 'rainy-11', 'rainy-12', 'rainy-13', 'rainy-14',
       'avg_rain2', 'Dry_Wet_Diff-1', 'Dry_Wet_Diff-2', 'Dry_Wet_Diff-3',
       'Dry_Wet_Diff-4', 'Dry_Wet_Diff-5', 'Dry_Wet_Diff-6', 'Dry_Wet_Diff-7',
       'avg_dry1', 'Dry_Wet_Diff-8', 'Dry_Wet_Diff-9', 'Dry_Wet_Diff-10',
       'Dry_Wet_Diff-11', 'Dry_Wet_Diff-12', 'Dry_Wet_Diff-13',
       'Dry_Wet_Diff-14', 'avg_dry2'],
      dtype='object')

In [27]:
# Keep only the columns we plan to model with
drop_list=list(weather.columns)
drop_list.remove('Date')
drop_list.remove('avg_dry1')
drop_list.remove('avg_dry2')
drop_list.remove('avg_rain1')
drop_list.remove('avg_rain2')
weather2=weather.drop(drop_list, axis=1)
weather2.columns

Index(['Date', 'avg_rain1', 'avg_rain2', 'avg_dry1', 'avg_dry2'], dtype='object')

In [31]:
weather2.head()

,Date,avg_rain1,avg_rain2,avg_dry1,avg_dry2
0,2007-05-01,NaN,NaN,NaN,NaN
1,2007-05-02,0.00,NaN,1.000000,NaN
2,2007-05-03,0.00,NaN,0.500000,NaN
3,2007-05-04,0.00,NaN,0.666667,NaN
4,2007-05-05,0.25,NaN,0.750000,NaN


In [28]:
# Save the cleaned, engineered data
dirname = '../Austin/'
weather2.to_csv(dirname + 'weather2.csv', index=False)